In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [7]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    ################################################ s', Q = env(s, a)
    ################################################ ~s', ~Q = env(s, ~a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, retuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, 
                                                                   labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels))
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=aQs2_logits, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a') # max
    ###################################################### ~Q(s,~a)= r # min
    ###################################################### ~Q(s,~a)= r + Q'(s',a') # max
    targetQs = rewards + (gamma * aQs2_logits)
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    aloss2 += -tf.reduce_mean(eQs_logits) # max
    return actions_logits, aloss, eloss, aloss2

In [8]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [10]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [12]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [14]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:22.0000 R:22.0000 rate:0.0440 aloss:0.6823 eloss:2.7296 aloss2:0.7847 exploreP:0.9978
Episode:1 meanR:21.0000 R:20.0000 rate:0.0400 aloss:0.6893 eloss:2.7431 aloss2:0.7915 exploreP:0.9959
Episode:2 meanR:27.3333 R:40.0000 rate:0.0800 aloss:0.6918 eloss:2.6936 aloss2:0.7770 exploreP:0.9919
Episode:3 meanR:26.0000 R:22.0000 rate:0.0440 aloss:0.6856 eloss:2.6618 aloss2:0.7622 exploreP:0.9898
Episode:4 meanR:22.8000 R:10.0000 rate:0.0200 aloss:0.6908 eloss:2.6248 aloss2:0.7582 exploreP:0.9888
Episode:5 meanR:20.8333 R:11.0000 rate:0.0220 aloss:0.6893 eloss:2.6284 aloss2:0.7625 exploreP:0.9877
Episode:6 meanR:20.7143 R:20.0000 rate:0.0400 aloss:0.6894 eloss:2.6330 aloss2:0.7560 exploreP:0.9857
Episode:7 meanR:19.7500 R:13.0000 rate:0.0260 aloss:0.6988 eloss:2.6433 aloss2:0.7600 exploreP:0.9845
Episode:8 meanR:19.6667 R:19.0000 rate:0.0380 aloss:0.7026 eloss:2.6379 aloss2:0.7421 exploreP:0.9826
Episode:9 meanR:20.5000 R:28.0000 rate:0.0560 aloss:0.6909 eloss:2.6053 aloss2:0.7

Episode:80 meanR:22.7407 R:28.0000 rate:0.0560 aloss:0.6807 eloss:2.0573 aloss2:0.5968 exploreP:0.8335
Episode:81 meanR:23.0000 R:44.0000 rate:0.0880 aloss:0.6825 eloss:2.0581 aloss2:0.5894 exploreP:0.8298
Episode:82 meanR:22.9157 R:16.0000 rate:0.0320 aloss:0.6810 eloss:2.0768 aloss2:0.6107 exploreP:0.8285
Episode:83 meanR:22.8095 R:14.0000 rate:0.0280 aloss:0.6879 eloss:2.0772 aloss2:0.6044 exploreP:0.8274
Episode:84 meanR:23.0000 R:39.0000 rate:0.0780 aloss:0.6871 eloss:2.0957 aloss2:0.6023 exploreP:0.8242
Episode:85 meanR:23.3023 R:49.0000 rate:0.0980 aloss:0.6960 eloss:2.0754 aloss2:0.6081 exploreP:0.8202
Episode:86 meanR:23.2299 R:17.0000 rate:0.0340 aloss:0.6860 eloss:2.0698 aloss2:0.6009 exploreP:0.8188
Episode:87 meanR:23.3750 R:36.0000 rate:0.0720 aloss:0.6823 eloss:2.0592 aloss2:0.6196 exploreP:0.8159
Episode:88 meanR:23.3933 R:25.0000 rate:0.0500 aloss:0.6862 eloss:2.0880 aloss2:0.6286 exploreP:0.8139
Episode:89 meanR:23.2778 R:13.0000 rate:0.0260 aloss:0.6844 eloss:2.1346 

Episode:159 meanR:39.9100 R:25.0000 rate:0.0500 aloss:0.6741 eloss:2.1123 aloss2:1.3075 exploreP:0.5885
Episode:160 meanR:39.9100 R:22.0000 rate:0.0440 aloss:0.6761 eloss:2.1067 aloss2:1.3220 exploreP:0.5873
Episode:161 meanR:40.5500 R:76.0000 rate:0.1520 aloss:0.6805 eloss:2.1046 aloss2:1.3277 exploreP:0.5829
Episode:162 meanR:40.6900 R:26.0000 rate:0.0520 aloss:0.6803 eloss:2.1253 aloss2:1.3390 exploreP:0.5814
Episode:163 meanR:40.7700 R:22.0000 rate:0.0440 aloss:0.6797 eloss:2.1160 aloss2:1.3209 exploreP:0.5802
Episode:164 meanR:41.0900 R:50.0000 rate:0.1000 aloss:0.6788 eloss:2.0917 aloss2:1.3398 exploreP:0.5773
Episode:165 meanR:41.2200 R:24.0000 rate:0.0480 aloss:0.6813 eloss:2.1075 aloss2:1.3629 exploreP:0.5759
Episode:166 meanR:41.4700 R:47.0000 rate:0.0940 aloss:0.6804 eloss:2.0898 aloss2:1.3634 exploreP:0.5733
Episode:167 meanR:42.3300 R:102.0000 rate:0.2040 aloss:0.6800 eloss:2.0938 aloss2:1.3911 exploreP:0.5676
Episode:168 meanR:42.7100 R:70.0000 rate:0.1400 aloss:0.6760 el

Episode:238 meanR:43.4400 R:46.0000 rate:0.0920 aloss:0.6800 eloss:1.9758 aloss2:1.7487 exploreP:0.4272
Episode:239 meanR:43.1200 R:21.0000 rate:0.0420 aloss:0.6709 eloss:1.9943 aloss2:1.7129 exploreP:0.4263
Episode:240 meanR:42.6900 R:12.0000 rate:0.0240 aloss:0.6819 eloss:1.9778 aloss2:1.7605 exploreP:0.4258
Episode:241 meanR:42.0700 R:46.0000 rate:0.0920 aloss:0.6777 eloss:1.9837 aloss2:1.7439 exploreP:0.4239
Episode:242 meanR:42.1200 R:84.0000 rate:0.1680 aloss:0.6773 eloss:2.0290 aloss2:1.7450 exploreP:0.4204
Episode:243 meanR:42.5000 R:77.0000 rate:0.1540 aloss:0.6794 eloss:1.9619 aloss2:1.7548 exploreP:0.4173
Episode:244 meanR:43.2800 R:95.0000 rate:0.1900 aloss:0.6766 eloss:1.9738 aloss2:1.7433 exploreP:0.4134
Episode:245 meanR:43.8100 R:103.0000 rate:0.2060 aloss:0.6799 eloss:1.9671 aloss2:1.7540 exploreP:0.4093
Episode:246 meanR:43.0700 R:13.0000 rate:0.0260 aloss:0.6736 eloss:1.9691 aloss2:1.7624 exploreP:0.4088
Episode:247 meanR:42.9800 R:10.0000 rate:0.0200 aloss:0.6844 el

Episode:317 meanR:37.8800 R:43.0000 rate:0.0860 aloss:0.6845 eloss:1.8842 aloss2:1.8525 exploreP:0.3208
Episode:318 meanR:37.0400 R:14.0000 rate:0.0280 aloss:0.6775 eloss:1.8659 aloss2:1.8769 exploreP:0.3204
Episode:319 meanR:36.5400 R:28.0000 rate:0.0560 aloss:0.6757 eloss:1.8710 aloss2:1.8348 exploreP:0.3195
Episode:320 meanR:36.5400 R:22.0000 rate:0.0440 aloss:0.6846 eloss:1.8721 aloss2:1.8580 exploreP:0.3189
Episode:321 meanR:36.3000 R:75.0000 rate:0.1500 aloss:0.6817 eloss:1.8594 aloss2:1.8524 exploreP:0.3166
Episode:322 meanR:35.7200 R:64.0000 rate:0.1280 aloss:0.6767 eloss:1.8454 aloss2:1.8753 exploreP:0.3146
Episode:323 meanR:35.5100 R:74.0000 rate:0.1480 aloss:0.6822 eloss:1.8588 aloss2:1.8706 exploreP:0.3124
Episode:324 meanR:35.9500 R:59.0000 rate:0.1180 aloss:0.6828 eloss:1.8580 aloss2:1.8708 exploreP:0.3106
Episode:325 meanR:36.2600 R:51.0000 rate:0.1020 aloss:0.6775 eloss:1.8508 aloss2:1.8825 exploreP:0.3090
Episode:326 meanR:35.9100 R:43.0000 rate:0.0860 aloss:0.6844 elo

...

Episode:3027 meanR:146.5000 R:145.0000 rate:0.2900 aloss:0.1331 eloss:1.4067 aloss2:1.1756 exploreP:0.0100
Episode:3028 meanR:146.0200 R:119.0000 rate:0.2380 aloss:0.1316 eloss:1.3954 aloss2:1.1822 exploreP:0.0100
Episode:3029 meanR:146.2200 R:152.0000 rate:0.3040 aloss:0.1347 eloss:1.4111 aloss2:1.1825 exploreP:0.0100
Episode:3030 meanR:146.3100 R:138.0000 rate:0.2760 aloss:0.1480 eloss:1.4064 aloss2:1.1668 exploreP:0.0100
Episode:3031 meanR:146.7300 R:169.0000 rate:0.3380 aloss:0.1287 eloss:1.3944 aloss2:1.1699 exploreP:0.0100
Episode:3032 meanR:145.4000 R:87.0000 rate:0.1740 aloss:0.1220 eloss:1.3902 aloss2:1.1881 exploreP:0.0100
Episode:3033 meanR:145.0300 R:99.0000 rate:0.1980 aloss:0.1247 eloss:1.3823 aloss2:1.1946 exploreP:0.0100
Episode:3034 meanR:144.3800 R:98.0000 rate:0.1960 aloss:0.1235 eloss:1.4076 aloss2:1.1829 exploreP:0.0100
Episode:3035 meanR:144.6500 R:151.0000 rate:0.3020 aloss:0.1450 eloss:1.4081 aloss2:1.1754 exploreP:0.0100
Episode:3036 meanR:145.0400 R:169.0000 r

Episode:3104 meanR:140.0500 R:218.0000 rate:0.4360 aloss:0.1354 eloss:1.3956 aloss2:1.1864 exploreP:0.0100
Episode:3105 meanR:139.7500 R:146.0000 rate:0.2920 aloss:0.1173 eloss:1.3817 aloss2:1.1889 exploreP:0.0100
Episode:3106 meanR:140.0900 R:133.0000 rate:0.2660 aloss:0.1305 eloss:1.3875 aloss2:1.1828 exploreP:0.0100
Episode:3107 meanR:140.0200 R:110.0000 rate:0.2200 aloss:0.1204 eloss:1.3810 aloss2:1.1901 exploreP:0.0100
Episode:3108 meanR:139.6700 R:111.0000 rate:0.2220 aloss:0.1251 eloss:1.3890 aloss2:1.1917 exploreP:0.0100
Episode:3109 meanR:139.1100 R:141.0000 rate:0.2820 aloss:0.1153 eloss:1.3828 aloss2:1.1917 exploreP:0.0100
Episode:3110 meanR:138.9400 R:97.0000 rate:0.1940 aloss:0.1435 eloss:1.3985 aloss2:1.1772 exploreP:0.0100
Episode:3111 meanR:139.0300 R:156.0000 rate:0.3120 aloss:0.1294 eloss:1.3802 aloss2:1.1832 exploreP:0.0100
Episode:3112 meanR:139.1700 R:134.0000 rate:0.2680 aloss:0.1266 eloss:1.3858 aloss2:1.1932 exploreP:0.0100
Episode:3113 meanR:138.6800 R:113.0000

Episode:3181 meanR:131.5200 R:111.0000 rate:0.2220 aloss:0.1221 eloss:1.3480 aloss2:1.1872 exploreP:0.0100
Episode:3182 meanR:131.6400 R:125.0000 rate:0.2500 aloss:0.1146 eloss:1.3485 aloss2:1.2045 exploreP:0.0100
Episode:3183 meanR:132.2500 R:155.0000 rate:0.3100 aloss:0.1218 eloss:1.3523 aloss2:1.2068 exploreP:0.0100
Episode:3184 meanR:131.5500 R:88.0000 rate:0.1760 aloss:0.1486 eloss:1.3817 aloss2:1.2363 exploreP:0.0100
Episode:3185 meanR:131.6800 R:121.0000 rate:0.2420 aloss:0.1135 eloss:1.3504 aloss2:1.1790 exploreP:0.0100
Episode:3186 meanR:131.1800 R:121.0000 rate:0.2420 aloss:0.1184 eloss:1.3508 aloss2:1.2034 exploreP:0.0100
Episode:3187 meanR:131.8600 R:140.0000 rate:0.2800 aloss:0.1191 eloss:1.3580 aloss2:1.1983 exploreP:0.0100
Episode:3188 meanR:131.1100 R:114.0000 rate:0.2280 aloss:0.1115 eloss:1.3534 aloss2:1.1839 exploreP:0.0100
Episode:3189 meanR:130.6900 R:127.0000 rate:0.2540 aloss:0.1223 eloss:1.3498 aloss2:1.1855 exploreP:0.0100
Episode:3190 meanR:130.6200 R:116.0000

Episode:3258 meanR:130.4400 R:107.0000 rate:0.2140 aloss:0.1010 eloss:1.3460 aloss2:1.1652 exploreP:0.0100
Episode:3259 meanR:130.4500 R:90.0000 rate:0.1800 aloss:0.0953 eloss:1.3442 aloss2:1.1639 exploreP:0.0100
Episode:3260 meanR:130.2700 R:132.0000 rate:0.2640 aloss:0.1012 eloss:1.3453 aloss2:1.1632 exploreP:0.0100
Episode:3261 meanR:130.1700 R:123.0000 rate:0.2460 aloss:0.1051 eloss:1.3420 aloss2:1.1825 exploreP:0.0100
Episode:3262 meanR:130.1300 R:136.0000 rate:0.2720 aloss:0.1047 eloss:1.3455 aloss2:1.1791 exploreP:0.0100
Episode:3263 meanR:130.1600 R:149.0000 rate:0.2980 aloss:0.1042 eloss:1.3452 aloss2:1.1833 exploreP:0.0100
Episode:3264 meanR:130.1300 R:154.0000 rate:0.3080 aloss:0.0982 eloss:1.3461 aloss2:1.1882 exploreP:0.0100
Episode:3265 meanR:129.7100 R:115.0000 rate:0.2300 aloss:0.1028 eloss:1.3613 aloss2:1.1715 exploreP:0.0100
Episode:3266 meanR:129.1500 R:121.0000 rate:0.2420 aloss:0.0985 eloss:1.3460 aloss2:1.1546 exploreP:0.0100
Episode:3267 meanR:129.0600 R:125.0000

Episode:3335 meanR:135.0900 R:182.0000 rate:0.3640 aloss:0.1068 eloss:1.3563 aloss2:1.1663 exploreP:0.0100
Episode:3336 meanR:134.7900 R:141.0000 rate:0.2820 aloss:0.1042 eloss:1.3483 aloss2:1.1649 exploreP:0.0100
Episode:3337 meanR:134.9900 R:143.0000 rate:0.2860 aloss:0.0983 eloss:1.3484 aloss2:1.1477 exploreP:0.0100
Episode:3338 meanR:135.6000 R:141.0000 rate:0.2820 aloss:0.1241 eloss:1.3512 aloss2:1.1633 exploreP:0.0100
Episode:3339 meanR:135.6600 R:155.0000 rate:0.3100 aloss:0.0901 eloss:1.3499 aloss2:1.1415 exploreP:0.0100
Episode:3340 meanR:135.2900 R:101.0000 rate:0.2020 aloss:0.0949 eloss:1.3399 aloss2:1.1548 exploreP:0.0100
Episode:3341 meanR:135.9300 R:147.0000 rate:0.2940 aloss:0.0883 eloss:1.3438 aloss2:1.1593 exploreP:0.0100
Episode:3342 meanR:135.9200 R:91.0000 rate:0.1820 aloss:0.0923 eloss:1.3490 aloss2:1.1587 exploreP:0.0100
Episode:3343 meanR:136.2800 R:139.0000 rate:0.2780 aloss:0.0984 eloss:1.3506 aloss2:1.1589 exploreP:0.0100
Episode:3344 meanR:136.0700 R:132.0000

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [24]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
